In [ ]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'CarPrice_Assignment.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
car_price = pd.read_csv(file_content_stream)
car_price.head()

In [ ]:
## creating a list to store the results
md1_coef = list()
md2_coef = list()

for i in range(0,1000):
    ## defining input and target variables
    X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']]
    Y = car_price['price']

    ## splitting data into training and testing datasets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    ## estimating lambda for lasso by CV with 5 folds
    lasso_cv = LassoCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], normalize = True, cv = 5).fit(X_train, Y_train)

    ## extracting the best lambda value via cross validation
    cv_lambda = lasso_cv.alpha_
    print('The estimated for the lasso model is', cv_lambda)

    ## building the lasso model
    lasso_md = Lasso(alpha = cv_lambda, normalize = True).fit(X_train, Y_train)
    lasso_md.coef_
    
    